# HVNM Tutorial 04: Startup Shear — NLSQ to NUTS

**Fit startup shear σ(t) with the Hybrid Vitrimer Nanocomposite Model**

Under constant shear rate $\dot{\gamma}$, the HVNM transient stress shows:
1. **Linear elastic ramp**: $\sigma \approx (G_P + G_E + G_D)\dot{\gamma}t$ at short times
2. **Stress overshoot**: TST kinetics accelerate bond exchange under load
3. **Steady state**: $\sigma_{ss} = G_D \cdot \dot{\gamma}/(k_d^D(1 + (\dot{\gamma}/k_d^D)^2))$

The overshoot height and position constrain **V_act** (stress-activation volume),
while the initial slope and steady state fix the moduli and rates.

## Dataset
PNAS 2022 Digital Rheometer Twin — startup at $\dot{\gamma} = 1$ s$^{-1}$

## Estimated Runtime
- NLSQ: ~30 s (ODE integration) | NUTS: ~3 min (FAST_MODE) / ~20 min (production)

## 1. Setup

In [ ]:
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax openpyxl
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models import HVNMLocal

jax, jnp = safe_import_jax()
verify_float64()

sys.path.insert(0, "../..")
from examples.utils.hvnm_tutorial_utils import (
    configure_hvnm_for_fit,
    get_bayesian_config,
    get_fast_mode,
    get_nlsq_values,
    get_output_dir,
    load_pnas_startup,
    plot_fit_comparison,
    plot_ppc,
    plot_trace_and_forest,
    print_convergence,
    print_parameter_table,
    save_figure,
    save_results,
    setup_style,
)

setup_style()
print(f"JAX {jax.__version__}, FAST_MODE: {get_fast_mode()}")
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

## 2. Load Data and Apply QC

In [ ]:
data = load_pnas_startup(gamma_dot=1.0, max_points=200)

print(data.summary())
print(f"\nApplied shear rate: {data.protocol_kwargs['gamma_dot']} 1/s")

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(data.x, data.y, '-', lw=1, color='darkorange')
ax.set_xlabel(data.x_label)
ax.set_ylabel(data.y_label)
ax.set_title(f'Startup Shear: $\\dot{{\\gamma}}$ = {data.protocol_kwargs["gamma_dot"]} 1/s')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Detect overshoot
y = data.y_masked
idx_peak = np.argmax(y)
sigma_peak = y[idx_peak]
t_peak = data.x_masked[idx_peak]

# Estimate steady state from last 20% of data
n_ss = max(10, len(y) // 5)
sigma_ss = np.mean(y[-n_ss:])
overshoot_ratio = sigma_peak / sigma_ss if sigma_ss > 0 else float('nan')

print(f"Peak stress: {sigma_peak:.4g} Pa at t = {t_peak:.4g} s")
print(f"Steady-state stress (est.): {sigma_ss:.4g} Pa")
print(f"Overshoot ratio: {overshoot_ratio:.3f}")
if overshoot_ratio > 1.05:
    print("Stress overshoot detected — V_act is identifiable from this data.")
else:
    print("No clear overshoot — V_act may not be well constrained.")

## 3. Configure HVNM and Fit (NLSQ)

Startup constrains 6 HVNM parameters:
- **G_P, G_E, G_D**: Network moduli (elastic ramp slope = sum of all)
- **nu_0**: BER attempt frequency (E-network relaxation time)
- **k_d_D**: D-network dissociation rate (sets D-network relaxation)
- **V_act**: Stress-activation volume (controls overshoot height and position)

In [ ]:
model = HVNMLocal(kinetics="stress", include_dissociative=True)
fit_params = configure_hvnm_for_fit(
    model,
    protocol="startup",
    overrides={
        "G_P": sigma_ss * 0.3,     # Fraction of steady-state from P-network
        "G_E": sigma_ss * 0.3,     # E-network contribution
        "G_D": sigma_ss * 0.5,     # D-network dominates at steady state
        "nu_0": 1e9,
        "k_d_D": 1.0 / t_peak if t_peak > 0 else 1.0,
        "V_act": 1e-4,             # Stress activation volume
        "T": 300.0,
        "phi": 0.0,
    },
)
print(f"Fittable: {fit_params}")

t0 = time.time()
model.fit(
    data.x_masked,
    data.y_masked,
    test_mode="startup",
    gamma_dot=data.protocol_kwargs['gamma_dot'],
    max_iter=3000,
)
print(f"NLSQ: {time.time() - t0:.1f} s")

nlsq_vals = get_nlsq_values(model, fit_params)
for p, v in nlsq_vals.items():
    print(f"  {p} = {v:.4g}")

In [ ]:
fig = plot_fit_comparison(data, model, title="HVNM Startup: NLSQ Fit")
save_figure(fig, "hvnm_04_startup_nlsq_fit.png")
plt.show()

## 4. Bayesian Inference (NUTS)

Startup ODE integration makes each likelihood evaluation expensive.
NUTS will take longer than algebraic protocols (flow curve, SAOS).

In [ ]:
bayes_cfg = get_bayesian_config()
print(f"Config: {bayes_cfg}")

t0 = time.time()
result = model.fit_bayesian(
    data.x_masked,
    data.y_masked,
    test_mode="startup",
    gamma_dot=data.protocol_kwargs['gamma_dot'],
    **bayes_cfg,
)
print(f"NUTS: {time.time() - t0:.1f} s")

## 5. Diagnostics and PPC

In [ ]:
print_convergence(result, fit_params)
print()
print_parameter_table(fit_params, nlsq_vals, result.posterior_samples)

In [ ]:
display_arviz_diagnostics(result, fit_params, fast_mode=get_fast_mode())

In [ ]:
fig = plot_ppc(
    data, model, result.posterior_samples, fit_params,
    title="HVNM Startup: Posterior Predictive Check",
)
save_figure(fig, "hvnm_04_startup_ppc.png")
plt.show()

## 6. Save Results

In [ ]:
save_results(
    get_output_dir("startup"), model, result,
    param_names=fit_params,
    extra_meta={
        "dataset": "PNAS_DRT",
        "protocol": "startup",
        "gamma_dot": data.protocol_kwargs['gamma_dot'],
    },
)

## What to Change for Your Data

1. **Shear rate**: Change `gamma_dot` in `load_pnas_startup()` or supply your own data
2. **V_act kinetics**: Set `kinetics="stretch"` for stretch-based TST if stress-based doesn't fit
3. **Multiple shear rates**: Run this notebook at several $\dot{\gamma}$ and compare V_act consistency
4. **Nanoparticles**: Set `phi > 0` for nanocomposites
5. **Temperature**: Update `T` override

## Troubleshooting

- **No overshoot in fit**: Increase `V_act` initial guess. V_act = 0 gives monotone approach to steady state
- **ODE solver divergence**: Reduce `gamma_dot` or increase `max_iter` in NLSQ
- **Slow NUTS**: Startup requires ODE at each evaluation. Use FAST_MODE for exploration, production for publication
- **V_act unconstrained**: If there's no overshoot in the data, V_act can't be identified. Fix it and fit the other 5 params
- **Elastic ramp too steep/shallow**: G_P + G_E + G_D controls the initial slope. Adjust initial guesses accordingly